### Import Packages

In [1]:
!pip install anchor-exp

In [2]:
import utils
import anchor_tabular
import sklearn
import sklearn.ensemble
import numpy as np

ModuleNotFoundError: ignored

### Load data and Learn blackbox model

In [ ]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = 'datasets/'
dataset = utils.load_dataset(
    'adult', balance=True, dataset_folder=dataset_folder, discretize=True)

In [ ]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(
    dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(
    dataset.labels_test, c.predict(dataset.test)))

### Get an Anchor for blackbox model

Create Explainer

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

Get an Anchor

In [ ]:
idx = 0
np.random.seed(1)
trg = dataset.test[idx]

exp = explainer.explain_instance(trg, c.predict, threshold=0.95)

print('Prediction:', explainer.class_names[c.predict(trg.reshape(1, -1))[0]])
print('Anchor:     %s' % (' AND '.join(exp.names())))
print('Precision:  %.2f' % exp.precision())
print('Coverage:   %.2f' % exp.coverage())

Test precision and coverage of the Anchor

In [ ]:
# Get test examples where the anchor applies
fit_anchor = np.where(np.all(
    dataset.test[:, exp.features()] == trg[exp.features()],axis=1))[0]

same_label = lambda x : c.predict(dataset.test[x]) == c.predict(trg.reshape(1, -1))

print('Anchor test precision: %.2f' % (np.mean(same_label(fit_anchor))))
print('Anchor test coverage: %.2f' % (
    fit_anchor.shape[0] / float(dataset.test.shape[0])))